# AdventureWorks - Medium

In [1]:
import $ivy.`org.apache.spark::spark-sql:3.4.0`

import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import org.apache.spark._
import org.apache.spark.sql._
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions.Window

val spark = {
    NotebookSparkSession.builder()
    .progress(false)
    .appName("app14-2")
    // .master("spark://192.168.31.31:7077")
    .master("local[*]")
    .config("spark.sql.warehouse.dir", 
            "hdfs://192.168.31.31:9000/user/hive/warehouse") 
    .config("spark.cores.max", "4") 
    .config("spark.executor.instances", "1") 
    .config("spark.executor.cores", "2") 
    .config("spark.executor.memory", "10g") 
    .config("spark.shuffle.service.enabled", "false") 
    .config("spark.dynamicAllocation.enabled", "false") 
    .config("spark.sql.catalogImplementation", "hive")
    .config("spark.sql.repl.eagerEval.enabled", "true")
    .config("spark.driver.allowMultipleContexts", "true")
    .getOrCreate()
}

Loading spark-stubs, spark-hive
Adding Hive conf dir /opt/hive/conf to classpath
Creating SparkSession


SLF4J: No SLF4J providers were found.
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See https://www.slf4j.org/codes.html#noProviders for further details.


Spark UI

import $ivy.$                                  


import org.apache.log4j.{Level, Logger}

import org.apache.spark._

import org.apache.spark.sql._

import org.apache.spark.sql.types._

import org.apache.spark.sql.functions._

import org.apache.spark.sql.expressions.Window


spark: SparkSession = org.apache.spark.sql.SparkSession@16ca0364

In [2]:
import spark.implicits._
def sc = spark.sparkContext
val hiveCxt = new org.apache.spark.sql.hive.HiveContext(sc)

import spark.implicits._

defined function sc
hiveCxt: sql.hive.HiveContext = org.apache.spark.sql.hive.HiveContext@100b49f8

In [3]:
// Credit to Aivean
implicit class RichDF(val ds:DataFrame) {
    def showHTML(limit: Int = 50, truncate: Int = 100) = {
        import xml.Utility.escape
        val data = ds.take(limit)
        val header = ds.schema.fieldNames.toSeq        
        val rows: Seq[Seq[String]] = data.map { row =>
          row.toSeq.map {cell =>
            val str = cell match {
              case null => "null"
              case binary: Array[Byte] => binary.map("%02X".format(_)).mkString("[", " ", "]")
              case array: Array[_] => array.mkString("[", ", ", "]")
              case seq: Seq[_] => seq.mkString("[", ", ", "]")
              case _ => cell.toString
            }
            if (truncate > 0 && str.length > truncate) {
              // do not show ellipses for strings shorter than 4 characters.
              if (truncate < 4) str.substring(0, truncate)
              else str.substring(0, truncate - 3) + "..."
            } else {
              str
            }
          }: Seq[String]
        }
    publish.html(s""" <table>
                <tr>
                 ${header.map(h => s"<th>${escape(h)}</th>").mkString}
                </tr>
                ${rows.map {row =>
                  s"<tr>${row.map{c => s"<td>${escape(c)}</td>" }.mkString}</tr>"
                }.mkString}
            </table>
        """)
    }
}

defined class RichDF

In [4]:
val cust_aw = hiveCxt.table("sqlzoo.CustomerAW")
val cust_addr = hiveCxt.table("sqlzoo.CustomerAddress")
val addr = hiveCxt.table("sqlzoo.Address")
val product = hiveCxt.table("sqlzoo.Product")
val order_det = hiveCxt.table("sqlzoo.SalesOrderDetail")
val order_head = hiveCxt.table("sqlzoo.SalesOrderHeader")
val prod_model = hiveCxt.table("sqlzoo.ProductModel")
val prod_model_prod = hiveCxt.table("sqlzoo.ProductModelProductDescription")
val prod_desc = hiveCxt.table("sqlzoo.ProductDescription")
val prod_cat = hiveCxt.table("sqlzoo.ProductCategory")

cust_aw: DataFrame = [customerid: int, namestyle: string ... 11 more fields]
cust_addr: DataFrame = [customerid: int, addressid: int ... 1 more field]
addr: DataFrame = [addressid: int, addressline1: string ... 5 more fields]
product: DataFrame = [productid: int, name: string ... 12 more fields]
order_det: DataFrame = [salesorderid: int, salesorderdetailid: int ... 4 more fields]
order_head: DataFrame = [salesorderid: int, revisionnumber: int ... 17 more fields]
prod_model: DataFrame = [productmodelid: int, name: string ... 1 more field]
prod_model_prod: DataFrame = [productmodelid: int, productdescriptionid: int ... 1 more field]
prod_desc: DataFrame = [productdescriptionid: int, description: string]
prod_cat: DataFrame = [productcategoryid: int, parentproductcategoryid: int ... 1 more field]

## 6.
A "Single Item Order" is a customer order where only one item is ordered. Show the SalesOrderID and the UnitPrice for every Single Item Order.

In [5]:
(order_det.filter(col("OrderQty")===1)
 .groupBy("SalesOrderID", "UnitPrice")
 .agg(count("SalesOrderDetailID").alias("n"))
 .filter(col("n")===1)
 .select("SalesOrderID", "UnitPrice")
 .showHTML())

SalesOrderID,UnitPrice
71936,20.99
71936,149.87
71936,158.43
71832,158.43
71796,31.58
71815,202.33
71858,54.89
71858,12.14
71935,24.29
71898,602.35


## 7.
Where did the racing socks go? List the product name and the CompanyName for all Customers who ordered ProductModel 'Racing Socks'.

In [6]:
(product.join(order_det, "ProductID")
 .join(order_head, "SalesOrderID")
 .join(cust_aw, "CustomerID")
 .join(prod_model.withColumnRenamed("Name", "ModelName"), 
       "ProductModelID")
 .filter(col("ModelName")==="Racing Socks")
 .select("CompanyName", "Name")
 .distinct()
 .orderBy("CompanyName")
 .showHTML())

CompanyName,Name
Eastside Department Store,"Racing Socks, L"
Essential Bike Works,"Racing Socks, L"
Remarkable Bike Store,"Racing Socks, L"
Remarkable Bike Store,"Racing Socks, M"
Riding Cycles,"Racing Socks, L"
Sports Products Store,"Racing Socks, M"
Sports Products Store,"Racing Socks, L"
The Bicycle Accessories Company,"Racing Socks, L"
The Bicycle Accessories Company,"Racing Socks, M"
Thrifty Parts and Sales,"Racing Socks, M"


## 8.
Show the product description for culture 'fr' for product with ProductID 736.

In [7]:
(product.join(prod_model_prod, "ProductModelID")
 .join(prod_desc, "ProductDescriptionID")
 .filter((col("ProductID")===736) && 
         (col("Culture").like("%fr%")))
 .select("Description")
 .showHTML())

Description
"Le cadre LL en aluminium offre une conduite confortable, une excellente absorption des bosses pou..."


## 9.
Use the SubTotal value in SaleOrderHeader to list orders from the largest to the smallest. For each order show the CompanyName and the SubTotal and the total weight of the order.

In [8]:
(order_head.join(order_det, "SalesOrderID")
 .join(product, "ProductID")
 .join(cust_aw, "CustomerID")
 .withColumn("Weight", col("Weight") * col("OrderQty"))
 .na.fill(0, Array("Weight"))
 .groupBy("CompanyName", "SubTotal")
 .sum("Weight")
 .withColumn("sum(Weight)", round($"sum(Weight)", 2))
 .orderBy(col("SubTotal").desc)
 .showHTML()
)

CompanyName,SubTotal,sum(Weight)
Action Bicycle Specialists,108561.83,1133911.56
Metropolitan Bicycle Supply,98278.69,679588.02
Bulk Discount Store,88812.86,34813.05
Eastside Department Store,83858.43,565638.72
Riding Cycles,78029.69,504095.33
Many Bikes Store,74058.81,744328.6
Instruments and Parts Company,63980.99,731576.77
Extreme Riding Supplies,57634.63,589939.11
Trailblazing Sports,41622.05,234328.12
Professional Sales and Service,39785.33,396843.63


## 10.
How many products in ProductCategory 'Cranksets' have been sold to an address in 'London'?

In [9]:
(product.join(prod_cat.filter(col("name")==="Cranksets"), 
              "ProductCategoryID")
 .join(order_det, "ProductID")
 .join(order_head, "SalesOrderID")
 .join(cust_aw, "CustomerID")
 .join(cust_addr, "CustomerID")
 .join(addr.filter(addr("City")==="London"), "AddressID")
 .groupBy()
 .count()
 .showHTML())

count
2


In [10]:
spark.stop()